In [125]:
import pandas as pd
from sklearn.decomposition import PCA 
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from imblearn.over_sampling import SMOTE
from numpy import loadtxt
from keras.models import Sequential
from keras.layers import Dense
import pandas as pd
from sklearn.model_selection import train_test_split
from keras import regularizers
from keras.layers.core import Dropout
from keras.callbacks import EarlyStopping
from sklearn.metrics import roc_auc_score
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from tensorflow.keras.regularizers import l1
import tensorflow as tf

In [27]:
train=pd.read_csv('train.csv')
test=pd.read_csv('test.csv')

In [44]:
X=train.iloc[:,2:]
y=train.iloc[:,1]
y.head()

0    0
1    0
2    0
3    0
4    1
Name: FRACASO, dtype: int64

In [45]:
varBasuras=['B11','B13','B14','B21','B23','B24','B46','B48',
            'B51','B52','B53','B54','B61','B62','B63','B64',
            'B65','B66','B67','B611','B85']
Xtrain=X.drop(columns=varBasuras)
X.head()
Xtest=test.drop(columns=varBasuras)
Xtest=Xtest.drop(columns='CODIGO_EMPRESA')
columnas=Xtest.columns

In [40]:
# Assuming 'Xtrain' and 'Xtest' are your NumPy arrays
scaler = StandardScaler()

# Fit and transform the training data
Xtrain = scaler.fit_transform(Xtrain)

# Transform the test data using the scaler fitted on the training data
Xtest = scaler.transform(Xtest)

# Convert the scaled data back to DataFrames (assuming you have column names)
columns = columnas
Xtrain = pd.DataFrame(Xtrain, columns=columns)
Xtest = pd.DataFrame(Xtest, columns=columns)

In [35]:
Xtest.head()

,B12,B22,B31,B32,B41,B42,B43,B44,B45,B47,B68,B69,B610,B612,B613,B81,B82,B83,B84
0,0.168363,-0.099483,1.111287,-0.475814,-0.512163,1.900444,0.955281,-0.400554,-0.771355,0.771355,0.498946,-0.261168,-0.134302,-0.780720,0.025704,-0.018293,-0.150224,0.363442,-0.203247
1,0.127996,-0.047951,1.112499,0.256981,-0.537126,-1.317306,0.942998,-0.409310,-1.217968,1.217968,-0.205152,-0.232546,-0.000448,-0.171388,0.108249,0.242454,-0.019638,0.993576,0.316543
2,-0.063161,0.138909,-0.259860,0.293544,0.306983,0.751520,0.225712,0.374091,0.069565,-0.069565,-0.031508,0.103740,-0.051708,-0.361341,0.157167,1.092730,0.376994,0.985942,0.122351
3,-0.121583,0.050664,-0.367988,0.440204,-0.655048,-1.185319,-2.643972,-1.320815,0.083608,-0.083608,-0.322346,-0.295401,-0.309789,1.263421,0.260650,1.758911,0.718318,1.580868,0.850126
4,0.035433,0.423080,0.071143,-0.391101,-0.126540,-0.116635,0.442852,0.212700,0.169848,-0.169848,-0.013674,-0.227961,-0.242566,0.882997,0.187902,-0.206098,-0.195730,-0.250701,-0.221177


In [255]:
model=Sequential()
model.add(Dense(64, input_dim=Xtrain.shape[1], activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(128, input_dim=Xtrain.shape[1], activation='relu'))
model.add(Dropout(0.1))
 # Added layer with the same regularization
model.add(Dense(1, activation='sigmoid'))
# Reports the AUC of a model outputting a probability.
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=['accuracy'])


In [248]:
# Assuming X is your feature matrix and y is your target variable
X_train, X_test, y_train, y_test = train_test_split(Xtrain, y, test_size=0.2, random_state=42)


In [280]:
smote = SMOTE(sampling_strategy=0.1,random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)


In [281]:
model.fit(X_train_resampled, y_train_resampled, epochs=15,validation_data=(X_test,y_test))

Epoch 1/15
48/48 [==============================] - 0s 5ms/step - loss: 0.0892 - accuracy: 0.9651 - val_loss: 0.7559 - val_accuracy: 0.9417
Epoch 2/15
48/48 [==============================] - 0s 4ms/step - loss: 0.0877 - accuracy: 0.9658 - val_loss: 0.7038 - val_accuracy: 0.9528
Epoch 3/15
48/48 [==============================] - 0s 4ms/step - loss: 0.0909 - accuracy: 0.9724 - val_loss: 0.7402 - val_accuracy: 0.9417
Epoch 4/15
48/48 [==============================] - 0s 4ms/step - loss: 0.0851 - accuracy: 0.9697 - val_loss: 0.7241 - val_accuracy: 0.9417
Epoch 5/15
48/48 [==============================] - 0s 4ms/step - loss: 0.0847 - accuracy: 0.9677 - val_loss: 0.7136 - val_accuracy: 0.9472
Epoch 6/15
48/48 [==============================] - 0s 4ms/step - loss: 0.0823 - accuracy: 0.9677 - val_loss: 0.6746 - val_accuracy: 0.9528
Epoch 7/15
48/48 [==============================] - 0s 4ms/step - loss: 0.0865 - accuracy: 0.9658 - val_loss: 0.8756 - val_accuracy: 0.9500
Epoch 8/15
48/48 [==

In [282]:
probs=model.predict(Xtest)
probs

18/18 [==============================] - 0s 2ms/step


array([[3.35252425e-03],
       [4.52339984e-02],
       [1.61910465e-03],
       [2.39717906e-06],
       [5.65869013e-05],
       [1.55675625e-02],
       [1.84669662e-02],
       [2.96407561e-05],
       [5.74911246e-05],
       [8.62884099e-06],
       [5.77913561e-05],
       [3.77839982e-10],
       [4.95550541e-13],
       [2.28417758e-03],
       [3.29174072e-05],
       [9.91949405e-12],
       [6.08720526e-04],
       [1.40120205e-03],
       [4.77439137e-08],
       [8.88150744e-03],
       [1.49395573e-03],
       [2.40050885e-03],
       [1.43046290e-01],
       [8.43069211e-06],
       [2.23966357e-08],
       [6.24440028e-04],
       [2.47283140e-03],
       [2.63769052e-05],
       [1.70264425e-08],
       [2.41411990e-03],
       [2.22293809e-02],
       [1.13425223e-04],
       [4.54830937e-04],
       [3.31803335e-07],
       [6.91607420e-04],
       [6.61531240e-02],
       [4.64512780e-03],
       [9.07211751e-03],
       [1.88500839e-04],
       [4.55934964e-02],


In [278]:

# Convert the NumPy array to a pandas DataFrame
probabilities_df = pd.DataFrame(probs)
probabilities_df.index = probabilities_df.index + 1
# Save the DataFrame to a CSV file
probabilities_df.to_csv('intento.csv', index=True)